<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/rule_ml_pair_pron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [340]:
pip install pymorphy2[fast]

## **Импорт**

In [341]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [342]:
from IPython.display import SVG, display, HTML

In [343]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [344]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [345]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [346]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

annotations.csv		hv-info_gepatit2.txt	     rakpobedim_ru2.txt
annotation_table.csv	hv-info_gepatit2.txt.1	     rakpobedim_ru2.txt.1
annotation_table.csv.1	neuroleptic_antidepr2.txt    sample_data
content			neuroleptic_antidepr2.txt.1
.shortcut-targets-by-id
MyDrive
.file-revisions-by-id
.Trash


In [347]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [348]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-25 16:49:30--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt.2’

neuroleptic_antidep 100%[===================>]  11.12M  45.3MB/s    in 0.2s    

2021-05-25 16:49:31 (45.3 MB/s) - ‘neuroleptic_antidepr2.txt.2’ saved [11655409/11655409]

--2021-05-25 16:49:31--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

In [349]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-25 16:49:34--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-25 16:49:34 ERROR 404: Not Found.

--2021-05-25 16:49:35--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100469 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv.2’

annotation_table.cs 100%[===================>]   2.00M  --.-KB/s    in 0.

In [350]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [351]:
posts = my_txt.split('----')

In [352]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [353]:
morph.parse('он')[0].normal_form

'он'

## **Обучение векторизатора**

In [354]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(morph.parse(i)[0].normal_form) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [355]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [356]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [357]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
!ls
# !rm annotation_table.csv.2 annotation_table.csv.1 annotation_table.csv annotations.csv

annotations.csv		neuroleptic_antidepr2.txt
annotation_table.csv	neuroleptic_antidepr2.txt.1
annotation_table.csv.1	neuroleptic_antidepr2.txt.2
annotation_table.csv.2	rakpobedim_ru2.txt
content			rakpobedim_ru2.txt.1
hv-info_gepatit2.txt	rakpobedim_ru2.txt.2
hv-info_gepatit2.txt.1	sample_data
hv-info_gepatit2.txt.2


In [358]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,2385.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,2386.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,2387.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,2388.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [359]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [360]:
# tok_list = []
# for sent in 
# for token in anns_df.HEAD:
#   try:
#     tok_list.append(ft.wv.__getitem__(token.lower()))
#   except KeyError:
#     tok_list.append(0)
#     print(token, 'here')
# anns_df['TOKEN_VECT'] = tok_list

In [361]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [362]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [363]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [364]:
anns_df.SENT_NUM[18]

1.0

In [365]:
# topics = list(set(df.TOPIC_NUM.values)) 
# for topic_num in topics:
#   # if topic_num == 2:
#   #   break
#   # извлекаем предложения топика
#   sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
#   for sent_num in sents_in_topic:
#     sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num)]

In [366]:
# vvv = 0
# with open('content/MyDrive/coref_fasttext60k_1.csv') as csvfile:
#   with open('new_coref_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     for new_row in csvfile:
#       vvv +=1
#       # print(i.split('\t'))
#       if len(new_row.split('\t')) > 9:
#         new_row = new_row.split('\t')[:9]
#       writer.writerow(new_row)
# vvv

In [367]:
# cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k_1.csv', delimiter=' ')
# cor_df

In [368]:
# cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [369]:
# cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
# cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
# cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
# cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]


Добавим размеченные данные, и посмотрим, что получилось.

In [370]:
# cor_df['NER'] = anns_df['NER']
# cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df['ANN_TOKEN'] = anns_df['TOKEN']
# cor_df

In [371]:
# cor_df.loc()

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [372]:
# df = anns_df.iloc[:15000,:]
df = anns_df.iloc[:17200,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [373]:
len(df.loc[df['NER'] == 1])

604

Из них местоимений (прономинальные анафоры).

In [374]:
len(df.loc[df['NER'] == 3])

117

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [375]:
len(df.loc[df['NER'] == 2])

230

In [376]:
len(df.loc[df['NER'].isin([2,3])])

347

In [377]:
len(df.loc[df['NER'].isin([1,2,3])])

951

In [378]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [379]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.74367255, 1.0672952, 0.4427513, -0.00718458...","[0.1934802, 0.9862153, 0.193794, 0.6324422, -0...","[0.18214528, 0.42482236, 0.25480556, -0.247987..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.6807331, 0.8204332, 0.24968927, -0.20220336...","[0.74367255, 1.0672952, 0.4427513, -0.00718458...","[0.18214528, 0.42482236, 0.25480556, -0.247987..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[0.50205547, 0.903761, 0.592246, -0.338901, 0....","[0.6807331, 0.8204332, 0.24968927, -0.20220336...","[0.18214528, 0.42482236, 0.25480556, -0.247987..."
21,лекарством,0,1,1.0,1.0,21.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[0.32014084, 0.23710169, 0.13242272, 0.1219887...","[0.30877045, 0.05210442, 0.40708372, 0.1741310...","[0.18214528, 0.42482236, 0.25480556, -0.247987..."
142,рисполепта,0,1,1.0,13.0,142.0,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,1,142.0,sing,"[0.62865883, 0.5999332, 0.2755363, -0.10313408...","[0.37717536, 0.82462984, 0.5385625, 0.27880532...","[0.18214528, 0.42482236, 0.25480556, -0.247987..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16961,капецитабина,0,20,0.0,23.0,372.0,anim,gent,masc,NaN,NOUN,obj,назначил,NaN,NaN,masc,VERB,1,372.0,sing,"[0.33226147, 0.7147286, -0.034476563, -0.35904...","[0.22429304, 1.1660023, -0.04803299, -0.346238...","[0.24512853, 0.364704, 0.18704905, -0.18924204..."
17147,гемцитабина,0,20,3.0,1.0,558.0,anim,gent,masc,NaN,NOUN,nmod,побочку,inan,accs,femn,NOUN,1,558.0,sing,"[0.20799102, 0.90808153, 0.08167018, -0.371151...","[1.075783, 0.22411518, -0.058686167, 0.2439490...","[-0.2368678, 0.34312898, 1.1441252, -0.1990969..."
17164,капецитабине,0,20,3.0,2.0,575.0,anim,loct,masc,NaN,NOUN,nmod,отзывов,inan,gent,masc,NOUN,1,575.0,sing,"[0.4403542, 0.72532654, -0.06516096, -0.178569...","[0.30951902, -0.20995915, 0.14135073, 0.609884...","[0.506883, 0.4411788, 0.23506606, 0.32862663, ..."
17182,гемцитабин,1,20,4.0,1.0,593.0,anim,nomn,masc,NaN,NOUN,nmod,тему,inan,accs,femn,NOUN,1,593.0,sing,"[0.34379128, 1.156924, -0.023522697, -0.431543...","[0.79581046, 0.7280067, -0.7401876, 0.08276815...","[0.12538674, 0.30265155, 0.016594496, 0.025974..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [380]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.74367255, 1.0672952, 0.4427513, -0.00718458...","[0.1934802, 0.9862153, 0.193794, 0.6324422, -0...","[0.18214528, 0.42482236, 0.25480556, -0.247987..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.28180775, 0.585976, -0.027220845, -0.006679...","[0.1934802, 0.9862153, 0.193794, 0.6324422, -0...","[0.31877407, 0.27384245, 0.23799655, -0.019877..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.3386204, 0.8115517, 0.24658626, 0.023461971...","[0.1934802, 0.9862153, 0.193794, 0.6324422, -0...","[0.16827986, 0.36521766, 0.16673455, -0.139714..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[0.39166826, 0.99277925, -0.0719932, -0.163770...","[0.14862487, 0.9987901, -0.6877503, 0.21326372...","[0.19538836, 0.24671851, 0.22614184, -0.117926..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.40354937, 0.5370169, 0.72131264, 0.01367176...","[0.1934802, 0.9862153, 0.193794, 0.6324422, -0...","[0.19538836, 0.24671851, 0.22614184, -0.117926..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.39650926, 0.6585794, 0.22466663, 0.09290536...","[0.1934802, 0.9862153, 0.193794, 0.6324422, -0...","[0.19896087, 0.2520365, 0.2642898, -0.02040538..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[0.15776056, 0.28668377, -0.5131789, -0.648180...","[0.15776056, 0.28668377, -0.5131789, -0.648180...","[0.19896087, 0.2520365, 0.2642898, -0.02040538..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[-0.009512476, 0.25214523, 1.0603454, 0.401625...","[0.46519738, 0.33663577, 0.826196, -0.2671354,...","[0.26705536, 0.36440584, 0.17336996, 0.0138553..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.46309364, 0.925598, -0.15441523, -0.414916,...","[0.46309364, 0.925598, -0.15441523, -0.414916,...","[0.3741008, 0.3369214, 0.08105631, -0.07942103..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[0.22230078, 0.7548038, 0.5806439, -0.7783884,...","[0.6454563, 1.3787891, -0.08284547, -0.8815415...","[0.3741008, 0.3369214, 0.08105631, -0.07942103..."


In [381]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [382]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [383]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [384]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [385]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [386]:
df.TOKEN[1]

'препарат'

In [387]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [388]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.511493,0.548062,0.940151
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing,0.555853,0.548062,0.940151
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.748514,0.872590,0.940151
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.780698,0.538442,0.940151
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.747382,0.597916,0.940151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17195,теперь,1,20,4.0,1.0,606.0,NaN,NaN,NaN,NaN,ADVB,advmod,назначили,NaN,NaN,NaN,VERB,0,0.0,NaN,0.481405,0.560032,0.914776
17196,нам,1,20,4.0,1.0,607.0,NaN,datv,NaN,1per,NPRO,iobj,назначили,NaN,NaN,NaN,VERB,0,0.0,plur,0.529800,0.560032,0.914776
17197,назначили,1,20,4.0,1.0,608.0,NaN,NaN,NaN,NaN,VERB,conj,пошел,NaN,NaN,masc,VERB,0,0.0,plur,0.679308,0.541322,0.914776
17198,пить,1,20,4.0,1.0,609.0,NaN,NaN,NaN,NaN,INFN,xcomp,назначили,NaN,NaN,NaN,VERB,0,0.0,NaN,0.611021,0.560032,0.914776


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [389]:
# df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
# df.POST_NUM = [int(i) for i in df['POST_NUM']]
# # df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
df.IS_ANSWER = [int(i) for i in df['TOPIC_NUM']]
# df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [390]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [391]:
df.loc[df['TOKEN_NUM'] == 344 ]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
344,рисполепта,1,1,2.0,4.0,344.0,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,1,344.0,sing,0.733511,0.477531,0.987734
918,вышло,2,2,6.0,3.0,344.0,NaN,NaN,neut,NaN,VERB,ROOT,вышло,NaN,NaN,neut,VERB,0,0.0,sing,0.537783,0.506262,0.985670
2212,в,3,3,5.0,8.0,344.0,NaN,NaN,NaN,NaN,PREP,case,использовании,inan,loct,neut,NOUN,0,0.0,NaN,0.477080,0.551901,0.891423
5897,и,4,4,6.0,7.0,344.0,NaN,NaN,NaN,NaN,CONJ,cc,выписывались,NaN,NaN,NaN,VERB,0,0.0,NaN,0.438114,0.520339,0.980574
6542,в,5,5,17.0,0.0,344.0,NaN,NaN,NaN,NaN,PREP,case,мозгу,inan,loc2,masc,NOUN,0,0.0,NaN,0.477080,0.457997,0.980574
8883,их,7,7,0.0,32.0,344.0,NaN,accs,NaN,3per,NPRO,obj,принимать,NaN,NaN,NaN,INFN,0,0.0,plur,0.452732,0.508339,0.983035
11513,отвратно,9,9,0.0,39.0,344.0,NaN,NaN,NaN,NaN,ADVB,advmod,перенесла,NaN,NaN,femn,VERB,0,0.0,NaN,0.568667,0.527573,0.946235
12936,будет,15,15,0.0,31.0,344.0,NaN,NaN,NaN,3per,VERB,advcl,проходить,NaN,NaN,NaN,INFN,0,0.0,sing,0.476435,0.609814,0.930205
15278,ковид,16,16,9.0,0.0,344.0,inan,nomn,masc,NaN,NOUN,conj,наименование,inan,nomn,neut,NOUN,0,0.0,sing,0.571122,0.507422,0.747537
16465,сказал,18,18,7.0,2.0,344.0,NaN,NaN,masc,NaN,VERB,acl:relcl,парень,anim,nomn,masc,NOUN,0,0.0,sing,0.455498,0.529717,0.867651


In [392]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,1,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.446344,0.490678,0.940151
6,знаю,1,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.535778,0.490678,0.940151
7,стоило,1,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.522965,0.490678,0.940151
8,ли,1,1,1.0,0.0,8.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.499507,0.456048,0.940151
9,мне,1,1,1.0,0.0,9.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.504008,0.456048,0.940151
10,писать,1,1,1.0,0.0,10.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.500269,0.456048,0.940151


In [393]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [394]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', 1, 1, 1.0, 1.0, 21.0, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.47826406359672546, 0.5794814825057983, 0.9401507377624512],
      dtype=object)

In [395]:
# sents_in_topic 
# topic_num
# df = df.reset_index()

In [396]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [397]:
def rule_eval(my_X, my_y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(my_X)):
    if my_X[i] != 0:
      if my_X[i] == my_y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif my_X[i] == 0:
      if my_X[i] == my_y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')


In [398]:
df.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)].iloc[30:40, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT


In [399]:
topics = list(set(df.TOPIC_NUM.values))
topics
# all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [400]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      # current_anaphors = sent.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)]
      current_anaphors = sent.loc[(df['POS'] == 5) &  (df['NER'] > 1)]
      # current_antecedents = sent.loc[(sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            ann_clusters.append(anfr[-5])
            # и класс, который определило правило(антецедента из предложения)
            my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)




[]
---
[]
---
0 0
0 604


In [401]:
rule_eval(my_clusters, ann_clusters)

0  true positive
0  false positive
0  true negative
0  false negative
---------
0  presicion
0  recall
0  f-score


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [402]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS',
 'NUMBER']

In [403]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1,  1])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [404]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct', 'gen2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'удобнее', 'иллюстрации', 'поиске', 'темам', 'посмотреть', 'случай',
        'описала', 'побочку', 'гемцитабина', 'прошлась'],
       dtype='object', length=2898),
 'HEAD_ANIMACY': Index(['inan', 'anim', 

In [405]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [406]:
df.loc[ (df['NER'] == 3.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
475,его,1,1,7.0,1.0,475.0,-1,3,0,1,5,11,199,0,2,2,1,3,2.0,1,0.538783,0.356303,0.983795
566,он,1,1,9.0,3.0,566.0,-1,0,0,1,5,12,231,-1,-1,0,0,3,536.0,1,0.518812,0.414909,0.983795
587,он,2,2,1.0,0.0,13.0,-1,0,0,1,5,12,235,-1,-1,0,0,3,11.0,1,0.518812,0.529281,0.983795
594,он,2,2,1.0,1.0,20.0,-1,0,0,1,5,12,242,-1,-1,-1,0,3,11.0,1,0.518812,0.549865,0.983795
736,него,2,2,3.0,5.0,162.0,-1,1,0,1,5,10,293,-1,-1,-1,0,3,108.0,1,0.526389,0.563216,0.982309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14906,него,15,15,19.0,3.0,2314.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,2295.0,1,0.526389,0.471126,0.975469
14967,он,16,16,1.0,4.0,33.0,-1,0,0,1,5,12,2504,-1,-1,0,0,3,0.0,1,0.518812,0.583091,0.975469
16328,они,18,18,6.0,2.0,207.0,-1,0,-1,1,5,12,2507,-1,-1,-1,9,3,0.0,0,0.446876,0.496836,0.951310
16391,его,18,18,6.0,5.0,270.0,-1,3,0,1,5,1,1588,-1,-1,-1,5,3,0.0,1,0.538783,0.545282,0.936769


In [407]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,1,1,0.0,0.0,2.0,1,0,0,-1,1,2,1,0,0,0,1,1,2.0,1,0.748514,0.872590,0.940151
3,рисполепт,1,1,0.0,0.0,3.0,0,1,1,-1,1,3,2,1,0,0,1,1,3.0,0,0.780698,0.538442,0.940151
4,сперидан,1,1,0.0,0.0,4.0,-1,-1,0,-1,2,4,3,0,1,1,1,1,4.0,1,0.747382,0.597916,0.940151
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.478264,0.579481,0.940151
142,рисполепта,1,1,1.0,13.0,142.0,0,0,1,-1,1,21,69,0,2,1,1,1,142.0,1,0.733511,0.513843,0.940151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16961,капецитабина,20,20,0.0,23.0,372.0,1,1,0,-1,1,1,1405,-1,-1,0,0,1,372.0,1,0.723284,0.585758,0.901807
17147,гемцитабина,20,20,3.0,1.0,558.0,1,1,0,-1,1,21,2895,0,2,1,1,1,558.0,1,0.631628,0.533001,0.642479
17164,капецитабине,20,20,3.0,2.0,575.0,1,4,0,-1,1,21,2277,0,1,0,1,1,575.0,1,0.733918,0.400140,0.758116
17182,гемцитабин,20,20,4.0,1.0,593.0,1,0,0,-1,1,21,121,0,2,1,1,1,593.0,1,0.615298,0.472652,0.893035


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [408]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)

In [409]:
train = train.reset_index()

In [410]:
test = test.reset_index()

In [411]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,11565,наркотики,9,9,0.0,50.0,396.0,0,3,0,-1,1,18,2121,-1,0,-1,11,2,0.0,0,0.449996,0.428758,0.817399
1,11924,таб,11,11,7.0,0.0,120.0,-1,-1,-1,-1,-1,2,2190,-1,-1,-1,-1,2,116.0,-1,0.638869,0.610794,0.963449
2,454,препарат,1,1,7.0,0.0,454.0,0,0,0,-1,1,12,189,-1,-1,-1,0,2,2.0,1,0.555853,0.500703,0.983795
3,10053,химия,7,7,8.0,8.0,1514.0,0,0,1,-1,1,12,1849,-1,-1,-1,0,2,0.0,1,0.563985,0.571254,0.962667
4,2263,их,3,3,5.0,12.0,395.0,-1,3,-1,1,5,1,571,-1,-1,-1,5,3,388.0,0,0.452732,0.666801,0.966394


In [412]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [413]:
# test.loc[test['NER'] != 0]

In [414]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

In [415]:
# best set

# X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
# train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

Делаем наборы для обучения моделей.

In [421]:
X, y, X_test, y_test = train.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
                                   'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',\
                                   	'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['TOKEN','GENDER', 'NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', \
                                      'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',\
                                      'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18],\
                                       test.COREFERENCE_CLUSTER

In [422]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [423]:
X.NER = [float(i) for i in X.NER]
X_test.NER = [float(i) for i in X_test.NER]


In [424]:
y

0        0.0
1        0.0
2      156.0
3      135.0
4        0.0
       ...  
272      0.0
273    564.0
274     58.0
275    135.0
276    806.0
Name: COREFERENCE_CLUSTER, Length: 277, dtype: float64

Обучаем модели.

In [425]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.6285714285714286

In [426]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.6428571428571429

In [427]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.6142857142857143

In [428]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)


1.0

In [429]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.4

In [430]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.6571428571428571

In [431]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [432]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [433]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [434]:
# my_eval(list(naive_pred), list(y_test))

In [435]:
my_eval(list(pred_knn), list(y_test))

20  true positive
22  false positive
0  true negative
0  false negative
---------
0.47619047619047616  presicion
1.0  recall
0.6451612903225806  f-score


In [436]:
my_eval(list(pred_entropy), list(y_test))

19  true positive
25  false positive
0  true negative
0  false negative
---------
0.4318181818181818  presicion
1.0  recall
0.6031746031746031  f-score


In [437]:
my_eval(list(pred), list(y_test))

18  true positive
26  false positive
0  true negative
0  false negative
---------
0.4090909090909091  presicion
1.0  recall
0.5806451612903226  f-score


In [438]:
my_eval(list(pred_randf), list(y_test))

17  true positive
27  false positive
0  true negative
0  false negative
---------
0.38636363636363635  presicion
1.0  recall
0.5573770491803279  f-score


In [439]:
my_eval(list(pred_regr), list(y_test))

3  true positive
67  false positive
0  true negative
0  false negative
---------
0.04285714285714286  presicion
1.0  recall
0.0821917808219178  f-score


In [440]:
my_eval(list(pred_svm), list(y_test))

2  true positive
20  false positive
0  true negative
0  false negative
---------
0.09090909090909091  presicion
1.0  recall
0.16666666666666669  f-score


In [441]:
# df['TOPIC_NUM'] == 1
# df.loc[df['TOPIC_NUM'] == 1]

In [442]:
df.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
         'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'HEAD_ANIMACY',	'HEAD_CASE',	\
         'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18]

,TOKEN_NUM,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,0.0,0,0.0,0.511493,0.940151
1,1.0,0,0.0,0.555853,0.940151
2,2.0,1,2.0,0.748514,0.940151
3,3.0,1,3.0,0.780698,0.940151
4,4.0,1,4.0,0.747382,0.940151
...,...,...,...,...,...
17195,606.0,0,0.0,0.481405,0.914776
17196,607.0,0,0.0,0.529800,0.914776
17197,608.0,0,0.0,0.679308,0.914776
17198,609.0,0,0.0,0.611021,0.914776


In [443]:
len(df.loc[(df['POS'] == 5) & (df['NER'] > 1)].TOKEN)

119

In [444]:
df.loc[(df['COREFERENCE_CLUSTER'] != 0) & (df['NER'] > 1)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
366,препарата,1,1,2.0,5.0,366.0,0,1,0,-1,1,21,154,0,1,1,1,2,344.0,1,0.508213,0.396940,0.987734
421,препарат,1,1,5.0,0.0,421.0,0,0,0,-1,1,0,1,0,0,0,1,2,2.0,1,0.555853,0.872590,0.987734
429,препарат,1,1,6.0,0.0,429.0,0,0,0,-1,1,12,179,-1,-1,-1,0,2,2.0,1,0.555853,0.528305,0.987734
446,последнему,1,1,6.0,2.0,446.0,-1,2,0,-1,10,7,184,-1,-1,-1,0,2,442.0,1,0.596424,0.515744,0.987734
454,препарат,1,1,7.0,0.0,454.0,0,0,0,-1,1,12,189,-1,-1,-1,0,2,2.0,1,0.555853,0.500703,0.983795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14427,него,15,15,14.0,2.0,1835.0,-1,1,0,1,5,10,195,-1,-1,-1,9,3,1827.0,1,0.526389,0.591696,0.787079
14440,его,15,15,14.0,2.0,1848.0,-1,3,0,1,5,1,1175,-1,-1,-1,0,3,1827.0,1,0.538783,0.502882,0.787079
14478,него,15,15,14.0,6.0,1886.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,1867.0,1,0.526389,0.471126,0.707177
14733,него,15,15,17.0,8.0,2141.0,-1,1,0,1,5,10,2278,-1,-1,1,0,3,2138.0,1,0.526389,0.597843,0.634580


# **ПОПАРНЫЙ МЕТОД**

In [445]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [446]:
topics = list(set(df.TOPIC_NUM.values))
topics


pair_anaphors = []
pair_antecedents = []
relation_list = []
counter = 0
ants_with_anaph = 0
len_anapfors = 0
for topic_num in topics:

  # all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] > 1)]
  all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

  all_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] == 1)]
  
  len_anapfors += len(all_anaphors)
  for anaphor in all_anaphors.values:
    # чтобы примеры влезли в память, и модель не переобучилась на антипримерах,
    # возьмем один положительный и два отрицательных класса для каждого анафора
    counter = 0
    is_relation = 0
    for antecedent in all_antecedents.values:
      

      # print(anaphor[-5], antecedent[5])
      if anaphor[-5] == antecedent[5]:
        ants_with_anaph += 1
        is_relation = 1
        # такие срезы, чтобы убрать лишние данные, и в том числе разметку с ответами
        pair_anaphors.append(list(anaphor)[3:6] )# + list(anaphor)[17:18]\
                             # + list(anaphor)[19:23])
        
        pair_antecedents.append(list(antecedent)[3:6]) # + list(antecedent)[17:18]\
                             # + list(antecedent)[19:23])
        
        relation_list.append(is_relation)
      else:
        if counter != 1:

          pair_anaphors.append(list(anaphor)[3:6] )# + list(anaphor)[17:18]\
                              # + list(anaphor)[19:23])
          
          pair_antecedents.append(list(antecedent)[3:6]) # + list(antecedent)[17:18]\
                              # + list(antecedent)[19:23])
          
          relation_list.append(is_relation)
          counter = 1

  # counter += 1
print(len_anapfors)
print('---')
print(pair_anaphors[0][0], pair_antecedents[0][0], relation_list[0])
len(pair_anaphors), len(pair_antecedents), len(relation_list)

725
---
1.0 0.0 0


(766, 766, 766)

In [447]:
# можно посмотреть, что получилось
relation_list[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [448]:
pair_anaphors[0]

[1.0, 0.0, 9.0]

In [449]:
# проверим, третьим элементом должен быть номер токена
len([i for i in relation_list if i==1])

88

In [450]:
# все сходится. мы взяли колонки с номерами поста, предложения и токена,\
# колонку NER, число, и вектора
df.loc[df.TOKEN_NUM == 21.0].iloc[[0]]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.478264,0.579481,0.940151


In [451]:
cosine = [cosine_similarity([pair_anaphors[i]], [pair_antecedents[i]])[0][0] for i in range(len(pair_anaphors))]
# pair_df = pd.DataFrame(columns=['anaphs', 'ants', 'rel'])
pair_df = pd.DataFrame(columns=['cosine', 'rel'])

# pair_df['anaphs'] = pair_anaphors
# pair_df['ants'] = pair_antecedents
pair_df['cosine'] = cosine
pair_df['rel'] = relation_list

print(pair_df.loc[pair_df['cosine'] > 1])
print(pair_df.loc[pair_df['rel'] > 1])
pair_df

Empty DataFrame
Columns: [cosine, rel]
Index: []
Empty DataFrame
Columns: [cosine, rel]
Index: []


,cosine,rel
0,0.993884,0
1,0.993127,0
2,0.995440,0
3,0.996729,0
4,0.997029,0
...,...,...
761,0.986387,0
762,0.986379,0
763,0.986649,0
764,0.986646,0


In [452]:
pair_df.loc[pair_df.rel != 0]

,cosine,rel
26,0.999889,1
27,0.999889,1
33,0.999994,1
36,0.999904,1
38,0.997928,1
...,...,...
643,0.999996,1
651,0.999984,1
669,0.999999,1
684,1.000000,1


In [453]:
train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
train = train.reset_index()
test = test.reset_index()


In [454]:
train.iloc[:1, 1:2]

,TOKEN_VECT
0,0.539772


In [455]:
test.iloc[:1, 1:2]

,TOKEN_VECT
0,0.591945


In [456]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [457]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
3  true positive
67  false positive
0  true negative
0  false negative
---------
0.04285714285714286  presicion
1.0  recall
0.0821917808219178  f-score


In [458]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
20  true positive
22  false positive
26  true negative
2  false negative
---------
0.47619047619047616  presicion
0.9090909090909091  recall
0.6249999999999999  f-score


In [459]:
dtclf = tree.DecisionTreeClassifier(max_depth=2000, )
dtclf.fit(X, y)


print('-----')
pred = dtclf.predict(X_test)
pair_eval(pred, y_test)


-----
19  true positive
25  false positive
26  true negative
0  false negative
---------
0.4318181818181818  presicion
1.0  recall
0.6031746031746031  f-score


In [460]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
20  true positive
24  false positive
26  true negative
0  false negative
---------
0.45454545454545453  presicion
1.0  recall
0.625  f-score


In [461]:
clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
rule_eval(pred, y_test)

-----
17  true positive
27  false positive
26  true negative
0  false negative
---------
0.38636363636363635  presicion
1.0  recall
0.5573770491803279  f-score


In [462]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [463]:
# clf = tree.DecisionTreeClassifier(max_depth=2000, )
# clf.fit(X, y)
# clf.score(X_test, y_test)

# **Извлечение номинативных анафоров**

In [464]:
df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,0,0,0,-1,1,1,0,-1,-1,-1,0,0,0.0,1,0.555853,0.548062,0.940151
16,проблем,1,1,1.0,1.0,16.0,0,1,1,-1,1,12,7,-1,-1,-1,3,0,0.0,0,0.527591,0.579481,0.940151
19,частности,1,1,1.0,1.0,19.0,0,4,1,-1,1,15,10,-1,-1,-1,4,0,0.0,1,0.500563,0.411267,0.940151
24,мере,1,1,1.0,2.0,24.0,0,2,1,-1,1,15,12,-1,-1,-1,4,0,0.0,1,0.585340,0.512961,0.940151
25,врач,1,1,1.0,2.0,25.0,1,0,0,-1,1,12,11,-1,-1,-1,0,0,0.0,1,0.639741,0.491005,0.940151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,дереала,2,2,4.0,2.0,239.0,0,1,0,-1,1,21,274,0,0,0,1,0,0.0,1,0.636992,0.469510,0.982309
849,месяц,2,2,5.0,1.0,275.0,0,3,0,-1,1,10,324,-1,-1,-1,5,0,0.0,1,0.567132,0.533752,0.985670
853,манию,2,2,5.0,1.0,279.0,0,3,1,-1,1,10,326,-1,-1,1,0,0,0.0,1,0.476588,0.564877,0.985670
856,мг,2,2,5.0,1.0,282.0,0,0,0,-1,1,10,326,-1,-1,1,0,0,0.0,1,0.571075,0.564877,0.985670


In [465]:
df.loc[(df['POS'] == 1) & (df['NER'] > 1)] 

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.478264,0.579481,0.940151
217,таблетки,1,1,1.0,21.0,217.0,0,3,1,-1,1,10,101,-1,-1,1,0,2,0.0,0,0.536826,0.702714,0.940151
295,таблетки,1,1,2.0,0.0,295.0,0,3,1,-1,1,10,101,-1,-1,1,0,2,0.0,0,0.536826,0.702714,0.987734
366,препарата,1,1,2.0,5.0,366.0,0,1,0,-1,1,21,154,0,1,1,1,2,344.0,1,0.508213,0.396940,0.987734
421,препарат,1,1,5.0,0.0,421.0,0,0,0,-1,1,0,1,0,0,0,1,2,2.0,1,0.555853,0.872590,0.987734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15630,прививки,16,16,16.0,0.0,696.0,0,1,1,-1,1,1,2636,1,0,0,1,2,0.0,1,0.617029,0.604673,0.918507
15651,вакцина,16,16,16.0,0.0,717.0,0,0,1,-1,1,21,2642,0,0,2,1,2,0.0,1,0.562525,0.492866,0.884891
15692,химиотерапии,16,16,17.0,3.0,758.0,0,1,1,-1,1,21,142,1,0,0,1,2,0.0,1,0.582360,0.591892,0.953768
15751,химиотерапий,16,16,18.0,1.0,817.0,0,1,1,-1,1,21,2609,0,1,1,1,2,0.0,0,0.642314,0.634780,0.950154


In [547]:
# df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :].append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)])
df.loc[(df['POS'] == 1) & (df['NER'] > 1)]
np.mean(df.loc[(df['POS'] == 1) & (df['NER'] > 1)].CASE)

1.1100478468899522

In [467]:
print(len(df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]))
df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]

19


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
446,последнему,1,1,6.0,2.0,446.0,-1,2,0,-1,10,7,184,-1,-1,-1,0,2,442.0,1,0.596424,0.515744,0.987734
3539,препарат.точнее,3,3,21.0,0.0,1671.0,-1,-1,-1,-1,13,0,691,-1,-1,-1,14,2,3.0,-1,0.614747,0.883487,0.794524
5222,новейший,3,3,38.0,5.0,3354.0,-1,0,0,-1,10,0,1085,-1,0,0,8,2,3350.0,1,0.628864,0.583122,0.980574
6857,тиапридал,5,5,23.0,6.0,659.0,-1,-1,0,-1,0,12,958,-1,-1,-1,0,2,659.0,1,0.717247,0.507526,0.980574
8834,это,7,7,0.0,25.0,295.0,-1,-1,-1,-1,3,12,941,-1,-1,-1,9,3,293.0,-1,0.466551,0.433009,0.983035
8928,хт,7,7,1.0,2.0,389.0,-1,-1,-1,-1,-1,21,1634,0,0,0,1,2,0.0,-1,0.581063,0.491617,0.983035
8987,хт,7,7,1.0,8.0,448.0,-1,-1,-1,-1,-1,21,1651,0,1,2,1,2,0.0,-1,0.581063,0.549501,0.983035
9086,хт,7,7,1.0,13.0,547.0,-1,-1,-1,-1,-1,10,229,-1,-1,-1,5,2,0.0,-1,0.581063,0.583721,0.983035
9497,ксф,7,7,1.0,57.0,958.0,-1,-1,-1,-1,-1,12,1791,-1,-1,-1,0,2,0.0,-1,0.519756,0.616502,0.980807
9935,хт,7,7,7.0,0.0,1396.0,-1,-1,-1,-1,-1,21,1634,0,0,0,1,2,0.0,-1,0.581063,0.491617,0.978946


In [577]:
med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:2409, :]\
            .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
            loc[:,['TOKEN_VECT', 'CASE', 'HEAD_VECT', 'NER']]


# med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:1409, :]\
#             .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
#             loc[:,['TOKEN_VECT', 'HEAD_VECT', 'SENT_VECT', 'NER']]

In [578]:
med_df

,TOKEN_VECT,CASE,HEAD_VECT,NER
1,0.555853,0,0.548062,0
16,0.527591,1,0.579481,0
19,0.500563,4,0.411267,0
24,0.585340,2,0.512961,0
25,0.639741,0,0.491005,0
...,...,...,...,...
15630,0.617029,1,0.604673,2
15651,0.562525,0,0.492866,2
15692,0.582360,1,0.591892,2
15751,0.642314,1,0.634780,2


In [589]:
train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
train = train.reset_index().drop(['index'], axis=1)
test = test.reset_index().drop(['index'], axis=1)
X, y, X_test, y_test = train.drop(['NER'], axis=1), \
                    train.NER, test.drop(['NER'], axis=1), test.NER

In [590]:
# pred = knn_clf.predict(X_test)
# pair_eval(pred, y_test)

In [591]:
X

,TOKEN_VECT,CASE,HEAD_VECT
0,0.413238,3,0.543120
1,0.485381,3,0.579044
2,0.484837,2,0.512961
3,0.548511,4,0.531576
4,0.609434,1,0.415718
...,...,...,...
1827,0.502560,0,0.453764
1828,0.623113,1,0.438414
1829,0.513533,0,0.445637
1830,0.515981,1,0.460780


In [592]:
y

0       2
1       0
2       0
3       0
4       0
       ..
1827    0
1828    0
1829    0
1830    0
1831    0
Name: NER, Length: 1832, dtype: int64

In [593]:
len([i for i in y if i == 2])

135

In [594]:
y_test

0      0
1      2
2      0
3      0
4      0
      ..
781    0
782    0
783    0
784    0
785    0
Name: NER, Length: 786, dtype: int64

In [595]:
len([i for i in y_test if i == 2])

72

In [596]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
786  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [597]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
29  true positive
6  false positive
708  true negative
43  false negative
---------
0.8285714285714286  presicion
0.4027777777777778  recall
0.5420560747663552  f-score


In [598]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
38  true positive
18  false positive
696  true negative
34  false negative
---------
0.6785714285714286  presicion
0.5277777777777778  recall
0.5937499999999999  f-score
